# Resume Q&A Agent

This notebook implements a multi-agent system using LangGraph to answer questions about candidate resumes. The system uses a Pinecone vector database to store and retrieve resume information.

In [9]:
import os
import operator

from dotenv import load_dotenv
from pydantic import SecretStr
from pinecone import Pinecone
from langgraph.types import Send
from IPython.display import Markdown
from langchain_core.documents import Document
from typing import Annotated, List, TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_pinecone import PineconeVectorStore
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage

# Load environment variables from .env file
load_dotenv()

True

In [10]:
# --- Environment and API Configuration ---

# Load API keys from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

if not OPENAI_API_KEY or not PINECONE_API_KEY:
    raise ValueError(
        "Please set your OPENAI_API_KEY and PINECONE_API_KEY environment " 
        "variables."
    )

# Configure the LLM and Embeddings model
MODEL_NAME = os.getenv('MODEL_NAME', 'gpt-4o')
EMBEDDING_MODEL = os.getenv('EMBEDDING_MODEL', 'text-embedding-3-small')
EMBEDDING_DIMENSIONS = int(os.getenv('EMBEDDING_DIMENSIONS', 768))

# Initialize the LLM for generation
llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0.1,
    api_key=SecretStr(OPENAI_API_KEY)
)

# Initialize the embeddings model
embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    dimensions=EMBEDDING_DIMENSIONS,
    api_key=SecretStr(OPENAI_API_KEY)
)

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

### Schemas and State

We define the data structures that will be used to manage the state of our graph.
- **`Candidates`**: A Pydantic model to structure the output of our planner, which identifies candidate names from the user's question.
- **`State`**: The main state of the graph, tracking the question, identified candidates, retrieved contexts, and the final answer.
- **`WorkerState`**: The state passed to each parallel worker, containing the specific candidate and the question.

In [11]:
# Schema for identifying candidates in the user's query
class Candidates(BaseModel):
    names: List[str] = Field(
        description="A list of candidate names mentioned in the user's query.",
    )

# LLM augmented with the schema for structured output
planner = llm.with_structured_output(Candidates)

# Graph state
class State(TypedDict):
    question: str
    candidates: List[str]
    context: Annotated[List[Document], operator.add]
    answer: str

# Worker state
class WorkerState(TypedDict):
    question: str
    candidate: str

### Graph Nodes

These are the core functions that will be executed as nodes in our graph.
- **`orchestrator`**: The entry point that uses an LLM to identify the candidates mentioned in the question.
- **`retrieval_worker`**: The worker function that runs in parallel for each candidate. It connects to the appropriate Pinecone index, retrieves relevant resume chunks, and formats them as context.
- **`responder`**: The final node that synthesizes the retrieved contexts from all workers into a single, coherent answer.

In [12]:
def orchestrator(state: State):
    """
    Identifies candidates from the user's question and updates the state.
    """
    print("Orchestrator: Identifying candidates...")
    question = state['question']    
    # Use the planner to extract candidate names
    candidate_model = planner.invoke(
        [
            SystemMessage(
                content=(
                    "You are an expert at extracting candidate names from a "
                    "user query."
                )
            ),
            HumanMessage(
                content=(
                    "Extract the full names of any candidates mentioned in the "
                    f"following query: '{question}'"
                )
            )
        ]
    )    
    candidates = candidate_model.names
    print(f"Orchestrator: Found candidates: {candidates}")    
    return {"candidates": candidates}


def retrieval_worker(state: WorkerState):
    """
    Retrieves context for a single candidate from their Pinecone index.
    """
    candidate = state['candidate']
    question = state['question']    
    print(f"Worker: Retrieving context for {candidate}...")    
    # Sanitize candidate name to create a valid index name
    index_name = f"{candidate.lower().replace(' ', '-')}-index"    
    try:
        # Ensure the index exists before trying to use it
        if index_name not in pc.list_indexes().names():
            print(f"Worker: Index '{index_name}' not found. Skipping.")
            return {"context": []}
        # Create a vector store instance
        vector_store = PineconeVectorStore.from_existing_index(
            index_name=index_name,
            embedding=embeddings,
        )        
        # Retrieve relevant documents
        retriever = vector_store.as_retriever(
            search_kwargs={
                'k': 3,
                'score_threshold': 0.35
            }
        )
        retrieved_docs = retriever.invoke(question)        
        print(f"Worker: Successfully retrieved context for {candidate}.")
        return {"context": retrieved_docs}
    except Exception as e:
        print(f"Worker: Error retrieving context for {candidate}: {e}")
        return {"context": []}


def responder(state: State):
    """
    Generates the final answer based on the retrieved contexts.
    """
    print("Responder: Generating final answer...")
    question = state['question']
    context = "".join(d.page_content for d in state['context'])    
    # Generate the final response
    system_message = (
        "You are a helpful assistant for a resume Q&A system. "
        "Based on the provided context from one or more resumes, answer the "
        "user's question accurately. "
        "If the context does not contain the answer, state that the "
        "information is not available in the resume(s)."
    )    
    human_message = f"Question: {question}\n\nContext:\n{context}"    
    response = llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content=human_message)
    ])    
    print("Responder: Done.")
    return {"answer": response.content}

### Graph Definition and Execution

Here, we build the graph by defining the nodes and the edges that connect them.
- The graph starts with the `orchestrator`.
- A conditional edge, `assign_workers`, dynamically creates parallel `retrieval_worker` tasks for each identified candidate.
- The `responder` node is called after all workers have completed.
- Finally, the graph is compiled and is ready to be invoked.

In [13]:
# Conditional edge function to assign work to parallel workers
def assign_workers(state: State):
    """
    Assigns a worker to each identified candidate.
    """
    print("Assigning work to workers...")
    # The `Send` tool allows us to trigger multiple parallel node executions
    return [
        Send("retrieval_worker", {"candidate": candidate, "question": state['question']})
        for candidate in state['candidates']
    ]

# Build the graph
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("orchestrator", orchestrator)
graph_builder.add_node("retrieval_worker", retrieval_worker)
graph_builder.add_node("responder", responder)

# Define edges
graph_builder.add_edge(START, "orchestrator")
graph_builder.add_conditional_edges("orchestrator", assign_workers, ["retrieval_worker"])
graph_builder.add_edge("retrieval_worker", "responder")
graph_builder.add_edge("responder", END)

# Compile the graph
resume_agent = graph_builder.compile()

### Running the Agent

Now you can ask questions about the resumes. The agent will identify the candidates, retrieve the relevant information from Pinecone, and generate an answer.

In [14]:
# --- Example 1: Question about a single candidate ---
question1 = "Did John Doe work at Microsoft?"
result1 = resume_agent.invoke({"question": question1})

Markdown(result1['answer'])

Orchestrator: Identifying candidates...
Orchestrator: Found candidates: ['John Doe']
Assigning work to workers...
Worker: Retrieving context for John Doe...
Orchestrator: Found candidates: ['John Doe']
Assigning work to workers...
Worker: Retrieving context for John Doe...
Worker: Successfully retrieved context for John Doe.
Responder: Generating final answer...
Worker: Successfully retrieved context for John Doe.
Responder: Generating final answer...
Responder: Done.
Responder: Done.


Yes. John Doe worked at Microsoft Corporation as a Senior Software Engineer from January 2023 to August 2025.

In [15]:
# --- Example 2: Question about multiple candidates ---
question2 = "Where did John Doe and Mark Nash work in August 2025?"
result2 = resume_agent.invoke({"question": question2})

Markdown(result2['answer'])

Orchestrator: Identifying candidates...
Orchestrator: Found candidates: ['John Doe', 'Mark Nash']
Assigning work to workers...
Worker: Retrieving context for John Doe...
Worker: Retrieving context for Mark Nash...
Orchestrator: Found candidates: ['John Doe', 'Mark Nash']
Assigning work to workers...
Worker: Retrieving context for John Doe...
Worker: Retrieving context for Mark Nash...
Worker: Successfully retrieved context for John Doe.
Worker: Successfully retrieved context for John Doe.
Worker: Successfully retrieved context for Mark Nash.
Responder: Generating final answer...
Worker: Successfully retrieved context for Mark Nash.
Responder: Generating final answer...
Responder: Done.
Responder: Done.


- John Doe: Microsoft Corporation (Senior Software Engineer), United States
- Mark Nash: DeepMind Technologies (Senior Machine Learning Engineer), United Kingdom

In [17]:
# --- Example 3: Question about a candidate not in the system ---
question3 = "What is Elon Musk's experience?"
result3 = resume_agent.invoke({"question": question3,})

Markdown(result3['answer'])

Orchestrator: Identifying candidates...
Orchestrator: Found candidates: ['Elon Musk']
Assigning work to workers...
Worker: Retrieving context for Elon Musk...
Orchestrator: Found candidates: ['Elon Musk']
Assigning work to workers...
Worker: Retrieving context for Elon Musk...
Worker: Index 'elon-musk-index' not found. Skipping.
Responder: Generating final answer...
Worker: Index 'elon-musk-index' not found. Skipping.
Responder: Generating final answer...
Responder: Done.
Responder: Done.


The information is not available in the provided resume(s).